[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/drarkadeep/dangerously-devilish-notebooks/blob/main/nma/fMRI/generating_reports/visualise_tasks.ipynb)

### You should run save-glm-result.ipynb in the parent directory to generate the csv files we are analysing here...

In [6]:
import os
import pandas as pd

def analyze_glm_results(result_dir):
    experiments = os.listdir(result_dir)
    complete_data = {}
    for experiment in experiments:
        conditions = os.listdir(f"{result_dir}/{experiment}")
        complete_data[experiment] = {}
        for condition in conditions:
            if not any(word in condition.lower() for word in ['drift', 'derivative', 'constant']):
                complete_data[experiment][condition] = pd.read_csv(f"{result_dir}/{experiment}/{condition}")

    all_data = []
    for experiment, conditions in complete_data.items():
        for condition, df in conditions.items():
            df['experiment_condition'] = f"{experiment}:{condition}"
            all_data.append(df)
    combined_df = pd.concat(all_data, ignore_index=True)

    with open('top_20_roi.md', 'w') as f:
        f.write("# Top 20 ROI for Each Experiment:Condition\n\n")
        for exp_cond in combined_df['experiment_condition'].unique():
            f.write(f"## {exp_cond}\n\n")
            top_20 = combined_df[combined_df['experiment_condition'] == exp_cond].nlargest(20, 't_statistic')[['ROI', 'region', 'network', 't_statistic']]
            f.write(top_20.to_markdown(index=False))
            f.write("\n\n")

    with open('network_averages.md', 'w') as f:
        f.write("# Average t-statistic Values by Network\n\n")
        for exp_cond in combined_df['experiment_condition'].unique():
            f.write(f"## {exp_cond}\n\n")
            network_averages = combined_df[combined_df['experiment_condition'] == exp_cond].groupby('network')['t_statistic'].mean().sort_values(ascending=False).reset_index()
            f.write(network_averages.to_markdown(index=False))
            f.write("\n\n")

    print("Analysis complete. Check 'top_20_roi.md' and 'network_averages.md' for results.")

result_dir = "../2nd_level_glm_result"
analyze_glm_results(result_dir)

Analysis complete. Check 'top_20_roi.md' and 'network_averages.md' for results.
